In [17]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import openpyxl
import plotly.express as px

In [13]:
class Data:
    def __init__(self, base_url='https://wdc.kugi.kyoto-u.ac.jp/dst_final/'):
        self.base_url = base_url

    def get_data_by_month(self, year, month):
        if(int(month)>9):
            url = f'{self.base_url}{year}{month}/index.html'
        else:
            url = f'{self.base_url}{year}0{month}/index.html'
        return url
    
    def get_data_by_year(self, year):
        url = f'{self.base_url}{year}/index.html'
        return url
    
    def scrapping_data(self, url):
        page = requests.get(url)
                
        soup = BeautifulSoup(page.content, 'html.parser')

        # Obtención de datos del texto
        data_text = soup.get_text()

        # Encontrar números enteros en el texto
        data_values = [int(value) for value in re.findall(r'-?\d+', data_text)]

        # Dividimos los datos en sublistas para cada día
        day_length = 24
        all_day_data = [data_values[i:i+day_length] for i in range(26, len(data_values), day_length+1)]

        dataframe = pd.DataFrame(all_day_data, index=range(1,len(all_day_data)+1), columns=range(1,len(all_day_data[0])+1))
        return dataframe


In [14]:
url_month = Data().get_data_by_month(2000, 1)
dataframe = Data().scrapping_data(url_month)
dataframe.describe()

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,...,31.000000,31.000000,31.000000,31.00000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,-21.967742,-20.903226,-20.451613,-20.419355,-20.774194,-19.096774,-17.903226,-16.548387,-15.870968,-17.096774,...,-16.322581,-14.129032,-13.935484,-14.16129,-14.870968,-16.580645,-18.354839,-19.258065,-20.354839,-20.903226
std,20.803019,19.655966,20.827608,21.149900,19.078277,18.237973,18.017315,17.668878,18.427772,18.857987,...,15.678833,17.940163,18.350178,16.74136,15.882363,18.231427,17.932370,19.522581,20.395340,20.221037
min,-97.000000,-92.000000,-96.000000,-97.000000,-79.000000,-65.000000,-66.000000,-62.000000,-63.000000,-69.000000,...,-54.000000,-55.000000,-57.000000,-47.00000,-44.000000,-65.000000,-73.000000,-81.000000,-86.000000,-92.000000
25%,-28.000000,-23.500000,-24.500000,-24.500000,-29.000000,-28.000000,-25.000000,-24.000000,-25.000000,-27.000000,...,-24.500000,-23.000000,-22.000000,-22.00000,-24.000000,-26.000000,-25.500000,-27.000000,-25.500000,-27.000000
50%,-19.000000,-20.000000,-17.000000,-17.000000,-18.000000,-18.000000,-17.000000,-17.000000,-16.000000,-17.000000,...,-18.000000,-16.000000,-14.000000,-17.00000,-16.000000,-16.000000,-16.000000,-15.000000,-15.000000,-19.000000
75%,-9.000000,-9.500000,-9.500000,-8.500000,-7.000000,-6.500000,-5.500000,-3.500000,-3.000000,-3.500000,...,-7.000000,-3.000000,-3.000000,-4.50000,-6.500000,-5.000000,-6.000000,-7.500000,-8.500000,-9.000000
max,3.000000,7.000000,16.000000,12.000000,9.000000,11.000000,10.000000,14.000000,15.000000,12.000000,...,12.000000,41.000000,46.000000,39.00000,34.000000,29.000000,9.000000,4.000000,4.000000,3.000000


In [27]:
# dataframes name columns
dataframe.column

AttributeError: 'DataFrame' object has no attribute 'column'

In [ ]:
fig = px.line